# Here you need to add the paper title, and then a small block of general explanation for a binder

# Notebook to generate dummy data to be used in the included notebooks

Import libraries

In [ ]:
import copy
import math
import collections
from scipy import interpolate as interp
from scipy import signal
import scipy
import pandas as pd
import glob
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget

# The cell below will need to be changed for a generalized binder

In [ ]:
sys.path.insert(0, '../ReSurfEMG')
from TMSiSDK.file_readers import Poly5Reader
import helper_functions as hf

# Reruns should be done from this cell as the start

Now you can pick a file from the list, which have been numbered.

In [ ]:
t_start = 0     # s
t_end = 7*60    # s
emg_sample_rate = 2048   # Hz
draeger_sample_rate = 100   # Hz 

# Ventilator settings:
peep = 10       # Positive end-expiratory pressure (cmH2O)
dp = 5          # Driving pressure (cmH2O)
rr = 22         # Respiratory rate /min
ie_ratio = 1/2  # Ratio between inspiratory and expiratory phase
ie_fraction = ie_ratio/(ie_ratio + 1)

# Patient variables
p_mus_max = 5   # Muscular pressure (cmH2O)
c = .050        # Respiratory system compliance (L/cmH2O)
r = 5           # Respiratory system resistance (cmH2O/L/s)
p_ip = -5       # Static interpleural pressure (cmH2O)

tau_mus_up = 0.3
tau_mus_down = 0.3

# To simulate occlussion measurements
# t_occs = np.array([])
t_occs = np.array([5*60+5, 5*60+21, 5*60+35])
t_occs = np.floor(t_occs*rr/60)*60/rr
for i, t_occ in enumerate(t_occs):
    if t_end < (t_occ + 60/rr):
        print('t=' +  str(t_occ) + ': t_occ should be more than a respiratory cycle from t_end\n')

t_occs

In [ ]:
# Ventilator data generator

# data_draeger_samples = data_draeger.samples[:data_draeger.num_samples]
# x_draeger = data_draeger_samples
y_draeger = np.array([i/draeger_sample_rate for i in range(int(t_start*draeger_sample_rate), int(t_end*draeger_sample_rate))])

# p_mus = 5*np.sin(y_draeger*rr/60*2*np.pi)
# p_mus[p_mus > 0] = 0
p_mus_block = (signal.square(y_draeger*rr/60*2*np.pi + 0.1, ie_fraction)+1)/2
for i, t_occ in enumerate(t_occs):
    n_occ = int(t_occ*draeger_sample_rate)    
    p_mus_block[n_occ:n_occ+int(draeger_sample_rate*60/rr)+1] = np.zeros((int(draeger_sample_rate*60/rr)+1,))
    #  p_mus_block[n_occ:n_occ+int(draeger_sample_rate*60/rr)+1] = -5*(signal.square(np.arange(int(draeger_sample_rate*60/rr)+1)*rr/60*2*np.pi, ie_fraction)+1)/2

pattern_gen_mus = np.zeros((len(y_draeger),))
for i in range(1, len(y_draeger)):
    if (p_mus_block[i-1]-pattern_gen_mus[i-1]) > 0:
        pattern_gen_mus[i] = pattern_gen_mus[i-1]+(p_mus_block[i-1]-pattern_gen_mus[i-1])/(tau_mus_up*draeger_sample_rate)
    else:
        pattern_gen_mus[i] = pattern_gen_mus[i-1]+(p_mus_block[i-1]-pattern_gen_mus[i-1])/(tau_mus_down*draeger_sample_rate)

p_mus = p_mus_max * pattern_gen_mus

p_block = dp * (signal.square(y_draeger*rr/60*2*np.pi, ie_fraction)+1)/2
# p_block = dp * (signal.square(y_draeger*rr/60*2*np.pi, 0.4)+1)/2
tau_dp_up = 10
tau_dp_down = 5
p_dp = -p_mus
for i in range(1, len(y_draeger)):
    if np.any((((t_occs*draeger_sample_rate)-i)<=0) &  ((((t_occs+60/rr)*draeger_sample_rate+1)-i)>0)):
        p_dp[i] = p_dp[i-1]+(-p_block[i-1]-p_dp[i-1])/tau_dp_up
    elif (p_block[i-1]-p_dp[i-1]) > 0:
        p_dp[i] = p_dp[i-1]+(p_block[i-1]-p_dp[i-1])/tau_dp_up
    else:
        p_dp[i] = p_dp[i-1]+(p_block[i-1]-p_dp[i-1])/tau_dp_down

p_vent = peep + p_dp

v_dot_vent = np.zeros((len(y_draeger),))
v_vent = np.zeros((len(y_draeger),))
for i in range(len(y_draeger)-1):
    if np.any((((t_occs*draeger_sample_rate)-i)<=0) &  ((((t_occs+60/rr)*draeger_sample_rate+1)-i)>0)):
        v_dot_vent[i+1] = 0
        v_vent[i+1] = 0
    else:
        v_dot_vent[i+1] = ((p_dp[i] + p_mus[i]) - v_vent[i] / c)/r
        v_vent[i+1] = v_vent[i] + v_dot_vent[i+1] * 1/draeger_sample_rate
    # if (p_block[i-1]-p_dp[i-1]) > 0:
    #     p_dp[i] = p_dp[i-1]+(p_block[i-1]-p_dp[i-1])/tau_dp_up
    # else:
    #     p_dp[i] = p_dp[i-1]+(p_block[i-1]-p_dp[i-1])/tau_dp_down


x_draeger = np.vstack((p_vent, v_dot_vent, v_vent))
data_draeger_samples = x_draeger



In [ ]:
# EMG generator

# data_emg_samples = data_emg.samples[:data_emg.num_samples]
# x_emg = data_emg_samples
y_emg = np.array([i/emg_sample_rate for i in range(int(t_start*emg_sample_rate), int(t_end*emg_sample_rate))])

# pattern_gen_emg = np.sin(y_emg*rr/60*2*np.pi)
# pattern_gen_emg[pattern_gen_emg < 0] = 0

emg_block = (signal.square(y_emg*rr/60*2*np.pi + 0.5, ie_fraction)+1)/2
for i, t_occ in enumerate(t_occs):
     n_occ = int(t_occ*emg_sample_rate)
     emg_block[n_occ:n_occ+int(emg_sample_rate*60/rr)+1] = (signal.square(np.arange(int(emg_sample_rate*60/rr)+1)*rr/60*2*np.pi, ie_fraction)+1)/2


pattern_gen_emg = np.zeros((len(y_emg),))
for i in range(1, len(y_emg)):
    if (emg_block[i-1]-pattern_gen_emg[i-1]) > 0:
        pattern_gen_emg[i] = pattern_gen_emg[i-1]+(emg_block[i-1]-pattern_gen_emg[i-1])/(tau_mus_up*emg_sample_rate)
    else:
        pattern_gen_emg[i] = pattern_gen_emg[i-1]+(emg_block[i-1]-pattern_gen_emg[i-1])/(tau_mus_down*emg_sample_rate)


x_emg = np.zeros((3,len(y_emg)))
part_ecg = np.zeros((len(y_emg),))
rng = np.random.default_rng(seed=42)
part_emg = pattern_gen_emg * np.random.normal(0, 0.5, size=(len(y_emg), ))
part_noise = np.random.normal(0, 0.5, size=(len(y_emg), ))
part_drift = np.zeros((len(y_emg),))
b, a  = scipy.signal.butter(4, 0.1, btype='low', analog=False, output='ba', fs=emg_sample_rate)
# part_drift = scipy.signal.filtfilt(b, a, np.random.normal(0, 0.5, size=(len(y_emg), )))

x_emg[0, :] = part_ecg + 0.05 * part_emg + 1 * part_noise + 20 * part_drift
x_emg[1, :] = part_ecg + 4 * part_emg + 1 * part_noise + 20 * part_drift
x_emg[2, :] = part_ecg + 8 * part_emg + 1 * part_noise + 20 * part_drift


data_emg_samples = x_emg


In [ ]:
fig, axis = plt.subplots(nrows=3, ncols=4, figsize=(10, 6), sharex='col')
axis[0, 0].grid(True)
axis[0, 0].plot(x_emg[0])
axis[0, 0].set(title='EMG leads: samples')
axis[1, 0].plot(x_emg[1])
axis[2, 0].plot(x_emg[2])

axis[0, 1].set(title='Draeger: samples')
axis[0, 1].grid(True)
axis[0, 1].plot(x_draeger[0])
axis[1, 1].plot(x_draeger[1])
axis[2, 1].plot(x_draeger[2])

axis[0, 2].grid(True)
axis[0, 2].plot(y_emg, x_emg[0])
axis[0, 2].set(title='EMG leads: seconds')
axis[1, 2].plot(y_emg, x_emg[1])
axis[2, 2].plot(y_emg, x_emg[2])

axis[0, 3].set(title='Draeger :seconds')
axis[0, 3].grid(True)
axis[0, 3].plot(y_draeger, x_draeger[0])
axis[1, 3].plot(y_draeger, x_draeger[1])
axis[2, 3].plot(y_draeger, x_draeger[2])